### NYC Taxi trips data
Our goal is to predict the number of NY City yellow taxi pickups in the next 1 hour for each of 257 pickup zones.  <br>

To do this, we will use Amazon Forecast to create forecasts with 1 hour frequency and 1 week forecast horizon.  For the demo, we'll use 3 months of historical data for training, and we will use the built-in feature of Amazon Forecast that reads in 14-day weather predictions as related data. <br>

<ul>
<li>Original data source for all open data from 2008-now:  <a href="https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page", target="_blank"> https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page</a> </li>
    <li>AWS-hosted location:  <a href="https://registry.opendata.aws/nyc-tlc-trip-records-pds/", target="_blank">https://registry.opendata.aws/nyc-tlc-trip-records-pds/ </a> </li>
    <li>AWS managed weather data ingestion as a service that is bundled with Amazon Forecast uses data source <a href="https://www.ecmwf.int/en/forecasts", target="_blank">European Centre for Medium-Range Weather Forecasts </a>, aggregated by location and by hour.  Initially only for USA and Europe, but depending on demand, possibly in the future weather will be available for other global locations. </li>
    </ul>
  
  
### Raw data gathering

The raw data is split 1 month per file, per Yellow, Green, or ForHire from 2008 through 2020, with each file around 600MB.  The entire raw data S3 bucket is HUGE.  <br>

I downloaded just 8 files:  yellow cabs July-Dec 2019, Jan-Feb 2020 to avoid COVID effects.  I processed them using AWS Glue and cleaned using Amazon Athena.<br>

### Data cleaning steps (done in Glue and Athena) 
<ul>
    <li>Calculate Trip duration in minutes</li>
    <li>Drop negative trip distances, 0 fares, 0 passengers, less than 1min trip durations </li>
    <li>Drop 2 unknown zones ['264', '265']
</ul>

In [1]:
import sys
import os

import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))

# get region from boto3
import boto3
REGION = boto3.Session().region_name

# # importing forecast notebook utility from notebooks/common directory
# sys.path.insert( 0, os.path.abspath("../../common") )
# # import util

import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = (15.0, 5.0)

pandas: 1.0.5
numpy: 1.19.1


### Read target time series required data

We will use the "Custom" schema.  See https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html.  <br>
Note: Domain-specific schemas are for convenience of wording, they have no impact on choice of algorithms or models. <br>

The documentation says the following fields are required for Custom schema:
<ul>
    <li>item_id (string) </li>
    <li>timestamp (timestamp) </li>
    <li>target_value (floating-point integer) </li>
    </ul>

Then your input .csv file must include the required columns.  The order of your columns does not matter, but you must have at least the required fields.  


![Image Select required columns for TTS](images/target_time_series_columns.png "Selecting required columns for Target Time Series")
<br>
We will include these required columns + create an additional geolocation "location" column required to use the built-in Amazon Forecast Weather Index.

In [2]:
# Read cleaned Target Time series data

df = pd.read_csv("data/TTS.csv")

# for demo purpose, drop geolocation and show how it was created
df.drop("location", axis=1, inplace=True)
print(df.shape)
print(df.dtypes)
print(f"num items:  {df.item_id.nunique()}")
start_time = df.timestamp.min()
end_time = df.timestamp.max()
print(f"start time: {start_time}")
print(f"end time: {end_time}")
df.sample(5)


(648414, 3)
timestamp       object
item_id          int64
actual_value     int64
dtype: object
num items:  260
start time: 2019-07-01 00:00:00
end time: 2020-02-29 23:00:00


timestamp  item_id  actual_value
328305  2019-10-28 14:00:00       76             3
125921  2019-08-14 19:00:00      211           151
367055  2019-11-12 05:00:00       93             7
257207  2019-10-01 21:00:00      244             9
513988  2020-01-07 11:00:00       48           226

In [3]:

## SINCE WE'RE GOING TO FORECAST AT HOURLY GRANULARITY, GO AHEAD AND AGGREGATE HOURLY

df = df.groupby(['timestamp', 'item_id']).sum()
df.reset_index(inplace=True)
print(f"df: {df.shape}")
df.sample(5)

df: (648414, 3)


timestamp  item_id  actual_value
434064  2019-12-07 17:00:00      254             1
382473  2019-11-18 01:00:00       43             1
325804  2019-10-27 14:00:00      237           434
341017  2019-11-02 09:00:00      211            41
393150  2019-11-22 04:00:00      125            11

<b>Geolocation data preparation steps</b>

Next, we need to create a geolocation column, to make our data location-aware.
<ul>
    <li>Convert shape files Latitude, Longitude to 0.2 accuracy required for weather data</li>
    <li>Join converted lat, lon data to raw trips data</li>
    <li>Create "location" column with required string format "lat_lon" </li>
    </ul>

In [4]:
## READ LAT, LON MAPPING FILE

latlon = \
    pd.read_csv("data/taxi_zones.csv")

print(latlon.shape)
print(latlon.dtypes)
latlon.head()

(263, 8)
X             float64
Y             float64
OBJECTID        int64
Shape_Leng    float64
Shape_Area    float64
zone           object
LocationID      int64
borough        object
dtype: object


X          Y  OBJECTID  Shape_Leng  Shape_Area  \
0 -74.176786  40.689516         1    0.116357    0.000782   
1 -73.826126  40.625724         2    0.433470    0.004866   
2 -73.849479  40.865888         3    0.084341    0.000314   
3 -73.977023  40.724152         4    0.043567    0.000112   
4 -74.189930  40.550340         5    0.092146    0.000498   

                      zone  LocationID        borough  
0           Newark Airport           1            EWR  
1              Jamaica Bay           2         Queens  
2  Allerton/Pelham Gardens           3          Bronx  
3            Alphabet City           4      Manhattan  
4            Arden Heights           5  Staten Island

### Convert raw lat lon to weather grids

In order to join ECMWF weather forecasts by geolocation, we need to convert our raw lat, lon values to 0.2 degree resolution.  See <a href= "https%3A%2F%2Fconfluence.ecmwf.int%2Fdisplay%2FFCST%2FIFS%2Bcycle%2B41r2%2Bresolution%2Bchanges"> https://confluence.ecmwf.int/display/FCST/IFS+cycle+41r2+resolution+changes </a> for more details.
<br>

In [5]:
# function to convert to closest 0.2 polygon geometries
def roundUpToNearestValue(theDF, colnames): 
    """converts input from decimal input to 0.2 polygon output
       returns dataframe with columns in colnames transformed and appended to end
    """
    localdf = theDF.copy()
    
    for c in colnames:
        localdf[f"{c}_nearest"] = round(localdf[c] * 5) / 5.0 
    
    return localdf

# test formula
# print(f"convert -73.977022: {round(-73.977022 * 5) / 5.0}")  #expect -74
# print(f"convert 40.724152: {round(40.724152 * 5) / 5.0}")    #expect 40.8

In [6]:
# add 0.2 degree resolved lat, lon

df_geo = roundUpToNearestValue(latlon, ['X', 'Y'])
df_geo.head()


X          Y  OBJECTID  Shape_Leng  Shape_Area  \
0 -74.176786  40.689516         1    0.116357    0.000782   
1 -73.826126  40.625724         2    0.433470    0.004866   
2 -73.849479  40.865888         3    0.084341    0.000314   
3 -73.977023  40.724152         4    0.043567    0.000112   
4 -74.189930  40.550340         5    0.092146    0.000498   

                      zone  LocationID        borough  X_nearest  Y_nearest  
0           Newark Airport           1            EWR      -74.2       40.6  
1              Jamaica Bay           2         Queens      -73.8       40.6  
2  Allerton/Pelham Gardens           3          Bronx      -73.8       40.8  
3            Alphabet City           4      Manhattan      -74.0       40.8  
4            Arden Heights           5  Staten Island      -74.2       40.6

### Join raw trips data to converted 0.2 degree resolution lat_lon data

In [8]:
## JOIN RAW TAXI DATA WITH LAT, LON FIELDS

joined = df.merge(df_geo, left_on="item_id", right_on="LocationID")
print(joined.shape, df.shape)
joined.sample(5)


(648414, 13) (648414, 3)


timestamp  item_id  actual_value          X          Y  \
3255    2019-11-15 13:00:00        4            16 -73.977023  40.724152   
520024  2019-12-26 17:00:00      261            90 -74.012919  40.708976   
200216  2020-01-09 16:00:00      113           131 -73.994628  40.732486   
413106  2019-08-08 17:00:00      226             4 -73.929321  40.735273   
422198  2019-12-22 03:00:00      229            58 -73.965174  40.756589   

        OBJECTID  Shape_Leng  Shape_Area                           zone  \
3255           4    0.043567    0.000112                  Alphabet City   
520024       261    0.027120    0.000034             World Trade Center   
200216       113    0.032745    0.000058        Greenwich Village North   
413106       226    0.168958    0.000762                      Sunnyside   
422198       229    0.042525    0.000095  Sutton Place/Turtle Bay North   

        LocationID    borough  X_nearest  Y_nearest  
3255             4  Manhattan      -74.0       40.8  
520024         261  Manhattan      -74.0       40.8  
200216         113  Manhattan      -74.0       40.8  
413106         226     Queens      -74.0       40.8  
422198         229  Manhattan      -74.0       40.8

<br>
In order to use the weather feature in Amazon Forecast, we require a geolocation "location" column with the specific format string "lat_lon". 
<br>

In [9]:

## CONCAT LAT_LON, WHERE "Y_NEAREST" = PICKUP_LATITUDE AND "X_NEAREST" = PICKUP_LONGITUDE

joined['location'] = joined.Y_nearest.astype(str) + "_" + joined.X_nearest.astype(str)
joined.head()


timestamp  item_id  actual_value          X          Y  OBJECTID  \
0  2019-07-01 00:00:00        4             9 -73.977023  40.724152         4   
1  2019-07-01 01:00:00        4             4 -73.977023  40.724152         4   
2  2019-07-01 02:00:00        4             2 -73.977023  40.724152         4   
3  2019-07-01 03:00:00        4             2 -73.977023  40.724152         4   
4  2019-07-01 04:00:00        4             8 -73.977023  40.724152         4   

   Shape_Leng  Shape_Area           zone  LocationID    borough  X_nearest  \
0    0.043567    0.000112  Alphabet City           4  Manhattan      -74.0   
1    0.043567    0.000112  Alphabet City           4  Manhattan      -74.0   
2    0.043567    0.000112  Alphabet City           4  Manhattan      -74.0   
3    0.043567    0.000112  Alphabet City           4  Manhattan      -74.0   
4    0.043567    0.000112  Alphabet City           4  Manhattan      -74.0   

   Y_nearest    location  
0       40.8  40.8_-74.0  
1       40.8  40.8_-74.0  
2       40.8  40.8_-74.0  
3       40.8  40.8_-74.0  
4       40.8  40.8_-74.0

In [10]:

## SELECT JUST THE COLUMNS YOU NEED FOR THE TARGET TIME SERIES

tts = joined[['timestamp', 'item_id', 'actual_value', 'location']].copy()
tts.head()


timestamp  item_id  actual_value    location
0  2019-07-01 00:00:00        4             9  40.8_-74.0
1  2019-07-01 01:00:00        4             4  40.8_-74.0
2  2019-07-01 02:00:00        4             2  40.8_-74.0
3  2019-07-01 03:00:00        4             2  40.8_-74.0
4  2019-07-01 04:00:00        4             8  40.8_-74.0

### Normally, you would save this tts file somewhere.  I've saved it already to data/TTS.csv

Now, let's do a quick visualization.

In [11]:
# Visualize items
random_items = [161, 162, 163, 170, 142]
fig, axs = plt.subplots(len(random_items), 1, figsize=(15, 15), sharex=True)

# force plot of just raw tts
temp = tts.loc[(tts.item_id.isin(random_items)), ['item_id', 'timestamp', 'actual_value']].copy()
temp.set_index('timestamp', inplace=True)
temp.columns = ['item_id', 'actual_value']
temp.head()

for i in range(len(random_items)):
    item = random_items[i]
    
    # zoomed-in plots
    zoomed = temp.loc[(temp['item_id']==item), :].copy()
#     zoomed = zoomed['2020-02-28':'2020-02-29']
    zoomed[['actual_value']].plot(ax=axs[i], title=f"Item_id={item}")
            
    axs[i].set_xlabel("Datetime")    #date
    fig.text(0.04, 0.5, 'Hourly demand', va='center', rotation='vertical')
    axs[i].grid(which='minor', axis='x')
    axs[i].grid(axis='y')